# Feed Forward Neural Net For Stock Price Movements Prediction 

In [ ]:
import sys
sys.path.append('../../')

In [ ]:
QUANDL_API_KEY = 'NEuvvy7yNkgc3xh1gNhj'

## Load Stock Price Data

In [ ]:
%%time
import datetime
from data_loading.stocks_data_loaders import get_stock_attribute_data
from data_loading.stocks_data_loaders import get_stock_data

tickers = ['WIKI/AAPL','WIKI/MSTF']

start_date_train=datetime.datetime(2013,1,1)

end_date_train=datetime.datetime(2017,12,31)

train_df=get_stock_data(tickers,start_date_train,end_date_train,QUANDL_API_KEY)

train_prices_df=get_stock_attribute_data(stocks_df=train_df,attribute='Adj. Close')

start_date_test=datetime.datetime(2018,1,1)

end_date_test=datetime.datetime(2018,12,31)

test_df=get_stock_data(tickers,start_date_test,end_date_test,QUANDL_API_KEY)

test_prices_df=get_stock_attribute_data(stocks_df=test_df,attribute='Adj. Close')

## Preprocess Prices For Training

In [ ]:
%%time

import numpy as np

from stock_movement_prediction.data_preprocessing import price_preprocessing

time_period=5

train_preprocessed_df=price_preprocessing(train_prices_df,time_period=time_period)
x_train=np.array(train_preprocessed_df['normalized_price_sequence'].tolist(),dtype=float)
y_train=np.array(train_preprocessed_df['onehot_encoded_price_movement'].tolist(),dtype=float)

test_preprocessed_df=price_preprocessing(test_prices_df,time_period=time_period)
x_test=np.array(test_preprocessed_df['normalized_price_sequence'].tolist(),dtype=float)
y_test=np.array(test_preprocessed_df['onehot_encoded_price_movement'].tolist(),dtype=float)

# Remove rows with NANs
is_nan=~np.isnan(x_train).any(axis=1)
x_train=x_train[is_nan]
y_train=y_train[is_nan]

is_nan=~np.isnan(x_test).any(axis=1)
x_test=x_test[is_nan]
y_test=y_test[is_nan]

## Train Neural Network

In [ ]:
%%time
import tensorflow as tf

from ml_models.neural_networks import create_feedworward_network_model

input_dim=x_train.shape[1]
hidden_layers_sizes=[100,100]
layers_activation='relu'
output_dim=y_train.shape[1]
output_activation='softmax'

model=create_feedworward_network_model(input_dim,
                                       hidden_layers_sizes,
                                       layers_activation,
                                       output_dim,
                                       output_activation)

optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

training = model.fit(x_train,y_train,epochs=1000,batch_size=16,validation_data=(x_test,y_test),shuffle=True)

## Performances Analysis

In [ ]:
import matplotlib.pyplot as plt

### Plot Training/Validation Metrics

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(8,2))
tag='loss'
axs[0].plot(training.history[f'{tag}'],label=f'training {tag}')
axs[0].plot(training.history[f'val_{tag}'],label=f'testing {tag}')
axs[0].legend()
axs[0].set_xlabel('training steps')
axs[0].set_ylabel(f'{tag}')
axs[0].grid()

tag='accuracy'
axs[1].plot(training.history[f'{tag}'],label=f'training {tag}')
axs[1].plot(training.history[f'val_{tag}'],label=f'testing {tag}')
axs[1].legend()
axs[1].set_xlabel('training steps')
axs[1].set_ylabel(f'{tag}')
axs[1].grid()
plt.show()

### Display Confusion Matrix